<a href="https://colab.research.google.com/github/Segrey372017/gb_ML_in_business/blob/main/ssmirnov_ml_business_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
!pip install catboost
!pip install scikit-uplift

In [ ]:
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels 
from sklearn.preprocessing import OneHotEncoder

#Домашняя работа к Уроку 4

### Задание 1 
<p> скачать набор данных маркетинговых кампаний отсюда <a href ="https://www.kaggle.com/davinwijaya/customer-retention"> Marketing Promotion Campaign Uplift Modelling </a></p>

In [ ]:
DATA_LINK   = 'https://raw.githubusercontent.com/Segrey372017/gb_ML_in_business/main/hw4_data.csv' # перенесен в мой гитхаб

In [ ]:
data = pd.read_csv(DATA_LINK)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


In [ ]:
data.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [ ]:
data.zip_code.value_counts()

Surburban    28776
Urban        25661
Rural         9563
Name: zip_code, dtype: int64

In [ ]:
data.channel.value_counts()

Web             28217
Phone           28021
Multichannel     7762
Name: channel, dtype: int64

In [ ]:
data.describe()

,recency,history,used_discount,used_bogo,is_referral,conversion
count,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000,64000.000000
mean,5.763734,242.085656,0.551031,0.549719,0.502250,0.146781
std,3.507592,256.158608,0.497393,0.497526,0.499999,0.353890
min,1.000000,29.990000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,64.660000,0.000000,0.000000,0.000000,0.000000
50%,6.000000,158.110000,1.000000,1.000000,1.000000,0.000000
75%,9.000000,325.657500,1.000000,1.000000,1.000000,0.000000
max,12.000000,3345.930000,1.000000,1.000000,1.000000,1.000000


In [ ]:
data.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel', 'offer', 'conversion'],
      dtype='object')

In [ ]:
data.isna().mean()

recency          0.0
history          0.0
used_discount    0.0
used_bogo        0.0
zip_code         0.0
is_referral      0.0
channel          0.0
offer            0.0
conversion       0.0
dtype: float64

### Задание 2
<p>поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.</p>

In [ ]:
data.columns = ['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel', 'treatment', 'target'] # изменяем названия колонок

In [ ]:
# для катбуста выберем категориальные колонки
cat_cols = list(data.dtypes[data.dtypes == 'object'].index)
cat_cols

['zip_code', 'channel', 'treatment']

In [ ]:
data.head(1)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0


In [ ]:
data['treatment'].value_counts() # проверим количество категорий

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: treatment, dtype: int64

In [ ]:
# выполним замену категорий на бинарные значения
tr_rep = {'No Offer':0, 'Buy One Get One':1, 'Discount': 1}

In [ ]:
data.treatment = data['treatment'].replace(tr_rep)

In [ ]:
data.treatment.value_counts() #Проверяем коректность замены, нулевых должно быть 21306

1    42694
0    21306
Name: treatment, dtype: int64

In [ ]:
# Поскольку Catboost что-то не хочет корректно работать с катеригориальными значениями, перекодирум в векторы соответсвующие колонки



In [ ]:
def OHEncoder(data):
    one_hot = OneHotEncoder(sparse = False)
    vect_cols = one_hot.fit_transform(data.values.reshape(-1,1))
    return pd.DataFrame(data = vect_cols, columns = list(*one_hot.categories_), index = data.index)

In [ ]:
zip_data = OHEncoder(data.zip_code)

In [ ]:
ch_data = OHEncoder(data.channel)

In [ ]:
data = pd.concat([data, zip_data, ch_data], axis = 1, join = 'inner')

In [ ]:
data.head(10)

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target,Rural,Surburban,Urban,Multichannel,Phone,Web
0,10,142.44,1,0,Surburban,0,Phone,1,0,0.0,1.0,0.0,0.0,1.0,0.0
1,6,329.08,1,1,Rural,1,Web,0,0,1.0,0.0,0.0,0.0,0.0,1.0
2,7,180.65,0,1,Surburban,1,Web,1,0,0.0,1.0,0.0,0.0,0.0,1.0
3,9,675.83,1,0,Rural,1,Web,1,0,1.0,0.0,0.0,0.0,0.0,1.0
4,2,45.34,1,0,Urban,0,Web,1,0,0.0,0.0,1.0,0.0,0.0,1.0
5,6,134.83,0,1,Surburban,0,Phone,1,1,0.0,1.0,0.0,0.0,1.0,0.0
6,9,280.20,1,0,Surburban,1,Phone,1,0,0.0,1.0,0.0,0.0,1.0,0.0
7,9,46.42,0,1,Urban,0,Phone,1,0,0.0,0.0,1.0,0.0,1.0,0.0
8,9,675.07,1,1,Rural,1,Phone,1,0,1.0,0.0,0.0,0.0,1.0,0.0
9,10,32.84,0,1,Urban,1,Web,1,0,0.0,0.0,1.0,0.0,0.0,1.0


In [ ]:
data.columns

Index(['recency', 'history', 'used_discount', 'used_bogo', 'zip_code',
       'is_referral', 'channel', 'treatment', 'target', 'Rural', 'Surburban',
       'Urban', 'Multichannel', 'Phone', 'Web'],
      dtype='object')

### Задание 3
<p> сделать разбиение набора данных не тренировочную и тестовую выборки</p>

In [ ]:
feats_col = ['recency', 'history', 'used_discount', 'used_bogo', 
       'is_referral', 'Rural', 'Surburban',
       'Urban', 'Multichannel', 'Phone', 'Web']
treat_col = ['treatment']
target_col = ['target']

In [ ]:
X = data[feats_col] # разделиь датасет на необходимые составляющие
y = data[target_col]
t = data[treat_col]

In [ ]:
# разбиваем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3,random_state = 25)

In [ ]:
type(X_train) # проверяем, что после разбиения на train/test данные остались в датафреймах, значить можно использовать индексацию

pandas.core.frame.DataFrame

In [ ]:
# разобьем признак коммуникации 
t_train = t.loc[X_train.index]
t_test = t.loc[X_test.index]

In [ ]:
# проверяем размерность выборок
print(f'X_train {X_train.shape}, X_test {X_test.shape}, Treat_train {t_train.shape}, Treat_test {t_test.shape}')
print(f'y_train {y_train.shape}, y_test {y_test.shape}')

X_train (44800, 11), X_test (19200, 11), Treat_train (44800, 1), Treat_test (19200, 1)
y_train (44800, 1), y_test (19200, 1)


In [ ]:
# проверим балансировку выборок
print(y_train.value_counts(normalize = True))
print(y_test.value_counts(normalize = True))

target
0         0.853214
1         0.146786
dtype: float64
target
0         0.853229
1         0.146771
dtype: float64


In [ ]:
# для катбуста выберем категориальные колонки
#cat_cols = list(X_train.dtypes[X_train.dtypes == 'object'].index)
#cat_cols

In [ ]:
#X_train[cat_cols] = X_train[cat_cols].astype('str')
#X_test[cat_cols] = X_test[cat_cols].astype('str')

In [ ]:
#X_train.info()

In [ ]:
#t_train.info()

### Задание 4
<p> провести uplift-моделирование 3 способами:<br>
A.одна модель с признаком коммуникации (S-learner)<br>
B.модель с трансформацией таргета<br>
C.вариант с двумя независимыми моделями<br>
 </p>

In [ ]:
# Создадим список моделей
models = ['S-learner', 'ClassTransformation', 'TwoModels']

In [ ]:
"""
# инициализируем модель CatBoost'а
boost_cls = list()
for i in range(0,4):
    boost_cls.append(CatBoostClassifier(iterations = 100, random_seed = 42, silent =True))
boost_cls
"""

"\n# инициализируем модель CatBoost'а\nboost_cls = list()\nfor i in range(0,4):\n    boost_cls.append(CatBoostClassifier(iterations = 100, random_seed = 42, silent =True))\nboost_cls\n"

In [ ]:
from lightgbm import LGBMClassifier

In [ ]:
boost_cls = list()
for i in range(0,4):
    boost_cls.append(LGBMClassifier(n_estimators = 100, random_state= 10))
boost_cls

[LGBMClassifier(random_state=10),
 LGBMClassifier(random_state=10),
 LGBMClassifier(random_state=10),
 LGBMClassifier(random_state=10)]

In [ ]:
# инициализируем модели uplift
S_learn = SoloModel(LGBMClassifier(n_estimators = 100, random_state= 10))
Ctransform = ClassTransformation(LGBMClassifier(n_estimators = 100, random_state= 10))
two_models = TwoModels(LGBMClassifier(n_estimators = 100, random_state= 10), 
                       LGBMClassifier(n_estimators = 100, random_state= 10))

In [ ]:
# обучаем модели на тренировочных данных
%%time
S_learn.fit(X_train, y_train, t_train)
Ctransform.fit(X_train, y_train, t_train)
two_models.fit(X_train, y_train, t_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), fo

ValueError: ignored

In [ ]:
lgbm = LGBMClassifier()

In [ ]:
lgbm.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier()

In [ ]:
lgbm.predict(X_test, y_test)

ValueError: ignored

In [ ]:
y_test.shape

(19200, 1)

In [ ]:
y_test.values